In [1]:
import sys  
import pandas as pd
from datetime import datetime

In [17]:
q_md_path = r'../data/variables/var-dia-Q_MEDIA_D-fill.zip'
q_mn_path = r'../data/variables/var-dia-Q_MN_D-fill.zip'
q_mx_path = r'../data/variables/var-dia-Q_MX_D-fill.zip'
cuenca_estacion_path = f'../data/stations/est_cuenca.csv'
q_md_out_path = r'../data/variables/var-mes-Q_MEDIA_D_MX_MIN-fill.csv'
q_md_cuenca_out_path = r'../data/matrix/cuenca_Q_MEDIA_M.csv'

In [3]:
dtparser = lambda x: datetime.strptime(x, '%Y-%m-%d')
usecols = ['CodigoEstacion','Fecha','ValorFilled']
indexcols = ['Fecha']
df_md = pd.read_csv(q_md_path, parse_dates = ['Fecha'], date_parser = dtparser, usecols = usecols, index_col = indexcols)
df_mn = pd.read_csv(q_mn_path, parse_dates = ['Fecha'], date_parser = dtparser, usecols = usecols, index_col = indexcols)
df_mx = pd.read_csv(q_mx_path, parse_dates = ['Fecha'], date_parser = dtparser, usecols = usecols, index_col = indexcols)

In [4]:
df_md = df_md['2000':'2019']
df_mn = df_mn['2000':'2019']
df_mx = df_mx['2000':'2019']

In [5]:
df_mn = df_mn.reset_index().set_index(['CodigoEstacion', 'Fecha'])
df_mx = df_mx.reset_index().set_index(['CodigoEstacion', 'Fecha'])
df_mn_mx = pd.concat([df_mn, df_mx], axis = 1)
df_mn_mx.columns = ['Q_MN_D', 'Q_MX_D']

In [6]:
df_mn_mx['Q_MEDIA_D'] = df_mn_mx.mean(axis = 1)

In [10]:
df_mn_mx = df_mn_mx.drop(['Q_MN_D','Q_MX_D'], axis = 1).dropna()

In [11]:
df_md = df_md.reset_index().set_index(['CodigoEstacion', 'Fecha'])
#df_mn_mx = df_mn_mx.reset_index().set_index(['CodigoEstacion', 'Fecha'])
df_md2 = pd.concat([df_md, df_mn_mx], axis = 1)

In [12]:
print(df_md.shape)
print(df_mn.shape)
print(df_mx.shape)
print(df_mn_mx.shape)
print(df_md2.shape)

(5361870, 1)
(3440655, 1)
(3440655, 1)
(2110676, 1)
(5361870, 2)


In [13]:
df_md2['ValorFilled2'] = df_md2['ValorFilled']
df_md2.loc[df_md2.ValorFilled.isna(), 'ValorFilled2'] = df_md2.loc[df_md2.ValorFilled.isna(), 'Q_MEDIA_D']
#df_md2[(df_md2.ValorFilled.notna()) & (df_md2.Q_MEDIA_D.isna())].head()

In [14]:
df_md2 = df_md2.drop(['ValorFilled','Q_MEDIA_D'], axis = 1).rename(columns = {'ValorFilled2': 'Q_MEDIA_D'})

In [15]:
df_md2 = df_md2.reset_index()
dfm = df_md2.groupby(['CodigoEstacion',df_md2.Fecha.dt.to_period('M')]).Q_MEDIA_D.median().reset_index()
dfm = dfm.rename(columns = {'CodigoEstacion': 'codigo'})

In [18]:
print(dfm.shape)
dfm.to_csv(q_md_out_path, index = False, date_format = '%Y-%m')

(176160, 3)


In [19]:
df_est_cuenca = pd.read_csv(cuenca_estacion_path, dtype = {'CUENCA':'string', 'Q_EST': 'string'}, usecols = ['CUENCA','Q_EST'])
df_est_cuenca = df_est_cuenca.rename(columns = {'CUENCA':'cuenca', 'Q_EST': 'codigo'})
df_est_cuenca = df_est_cuenca.dropna().groupby(['cuenca','codigo']).count().reset_index()
df_est_cuenca['codigo'] = df_est_cuenca['codigo'].astype(int)

In [21]:
dfm = dfm.merge(df_est_cuenca, on='codigo', how = 'inner' )
print(dfm.shape)
dfm.head()

In [26]:
dfm.to_csv(q_md_cuenca_out_path, index = False, date_format = '%Y-%m')